In [29]:
# Load required libraries
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist
import random
import pandas as pd

In [30]:
# Randomly generate 2 tables of latitudes and longitudes which match up to a small random shift
# We want to join these tables based on comparing the lat/longs using bipartite matching then see how accurate the match is. 
table_A = []
table_B = []
for i in range(1000):
    table_A_lat = random.uniform(-90, 90)
    table_A_long = random.uniform(-180, 180)
    table_A.append({'table_A_id':i,'lat':table_A_lat,'long': table_A_long})
    table_B.append({'table_A_id':i,'lat':table_A_lat + random.uniform(-0.25, 0.25),'long': table_A_long + random.uniform(-0.25, 0.25)})

In [31]:
# Create pandas dataframes.
table_A = pd.DataFrame.from_records(table_A)
random.shuffle(table_B)
table_B = pd.DataFrame.from_records(table_B)

In [32]:
# One degree of latitude is not equal to one degree of longitude, so account for that by scaling them independently.
# This is important when dealing with actual lat/long values.
table_A['lat_miles'] = table_A['lat'] * 69
table_B['lat_miles'] = table_B['lat'] * 69
table_A['long_miles'] = table_A['long'] * 54.6
table_B['long_miles'] = table_B['long'] * 54.6

In [33]:
# Create matrix of distances which will be used as weights for the bipartite matching.
distances = cdist(table_A[['lat_miles', 'long_miles']], table_B[['lat_miles','long_miles']])

In [34]:
# Get the predicted matches from bipartite matching.
assignment = linear_sum_assignment(distances)

In [35]:
# Reorder table_B according to the predictions, then assign the predicted index. 
# This should match the original index from table_A.
table_B = table_B.reindex(assignment[1])
table_B['predicted_index'] = assignment[0]

In [36]:
# Check the proportion matched correctly.
num_correct_matches = sum(table_B['table_A_id'] == table_B['predicted_index'])
fraction_correct = num_correct_matches / len(table_B)
print(fraction_correct)

1.0


In [37]:
# Observe any mistakes and note how close they are. 
table_B[table_B['table_A_id'] != table_B['predicted_index']]

,table_A_id,lat,long,lat_miles,long_miles,predicted_index
